# Kriging method

In [ ]:
# PYTHON STANDARD LIBRARY
import json
import io
import zipfile

from requests import get, post, put, delete
from requests.auth import HTTPBasicAuth

from pprint import pprint

# 3RD PARTY DATA SCIENCE
import pandas as pd
import numpy as np 

# GEOSPATIAL
import geopandas as gpd
import shapely
from shapely import wkt
import fiona
import seaborn as sns

from dateutil.relativedelta import *
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *
import pyproj    
import shapely
import shapely.ops as ops
from shapely.geometry.polygon import Polygon, Point
from functools import partial
import matplotlib.pyplot as plt
import random
from scipy.spatial import distance as dt
import mapclassify
import tqdm
import os

from matplotlib import pyplot
from shapely.geometry.polygon import LinearRing, Polygon
from sklearn.metrics import mean_absolute_percentage_error

---
# Import data from API

We are only using the API in this notebook for getting date/time list for the training and validation data, not the shapefiles themselves.

For getting the actual data please use the `aoi_data_by_time` notebook.

In [ ]:
token = input('Token >>  ');

In [ ]:
# SET THE API URL AS A LOCAL VARIABLE (IT WILL BE USED A LOT!)
api = 'https://go-services.orbitalinsight.com/api/v2/go'
# DEFINE THE HEADERS
headers = {"Content-Type": "application/json", 
           "X-Orbitalinsight-Auth-Token": token}
# DEFINE THE PROJECT AND VERSION IDS
pid = 'DzRmN00uWW-210911'
vid = 'DzRmN00uWW-210911-1.0.0'
# MODIFY THE API URL
vid_url = f'{api}/projects/{pid}/versions/{vid}'

# PERFORM THE GET COMMAND
response = get(vid_url, headers = headers)
# GENERATE A DICT AND CONVERT TO JSON
data_dict = {"detail": "false"}
data_json = json.dumps(data_dict)

# MODIFY THE API URL
dl_request_url = f'{api}/projects/{pid}/versions/{vid}/results/download'

# PERFORM THE POST COMMAND
response = post(dl_request_url, data = data_json, headers = headers)

# EXTRACT THE DOWNLOAD ID FROM THE RESPONSE 
download_id = response.json()['data']['results_download_id']
# MODIFY THE API URL
dl_url = f'{api}/projects/{pid}/versions/{vid}/results/download?results_download_id={download_id}' 

# PERFORM THE GET COMMAND
response = get(dl_url, headers = headers)

# GET THE DOWNLOAD URL IF THE DOWNLOAD IS READY
if 'Result is ready' in response.json()['data']['message']:
    download_url = response.json()['data']['url']
# PERFORM THE GET COMMAND
resp = get(download_url)
# PARSE THE ZIP DATA FROM THE HTTP RESPONSE INTO A PANDAS DATAFRAME
with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
    with z.open(z.namelist()[0]) as f:
        results = pd.read_csv(f)

### Filter out days with high cloud coverage

In [ ]:
def rule_out(level):
    """
    Filter the clouds with less than a certain cloud coverage `level`

    Returns: DataFrame object
    """
    results_low_cloud_coverage = results[(results["OI Cloud Coverage"] < level)]
    return results_low_cloud_coverage

In [ ]:
# Get all the date time with cloud coverage less than 0.1
data = rule_out(0.1)
# Name list for all AOI
aoi_list = rule_out(0.1)['AOI name'].unique()
aoi_list

In [ ]:
# Of all the dates with cloud coverage less than 0.1, we use those with cloud coverage less than 0.01
data['imaging_date'] = data['Measurement Timestamp (UTC)'].str.split(pat=' ', expand=True)[0]
data = data[data['AOI name'] == 'Aktubinsk']
train = list(data[data['OI Cloud Coverage'] > 0.01]['Measurement Timestamp (UTC)'].unique())
validation = list(data[data['OI Cloud Coverage'] <= 0.01]['Measurement Timestamp (UTC)'].unique())

In [ ]:
# Set a dictionary for all AOIs and their corresponding training/validaton dates
train = {}
validation = {}
for aoi in aoi_list:
    data = rule_out(0.1)
    data[data['AOI name'] == aoi]
    train[aoi] = list(data[data['OI Cloud Coverage'] > 0.01]['Measurement Timestamp (UTC)'].unique())
    validation[aoi] = list(data[data['OI Cloud Coverage'] <= 0.01]['Measurement Timestamp (UTC)'].unique())

---
# Functions

### Extract training and validation data (geometries & detections)

The following cells assume all of the AOI and detections shapefiles (cloud_cover and detections) are stored in a directory called `aoi_data_by_time`. Within this directory, they are sorted into folders named with the date that an observation occurs, and the final directory is the name of the AOI at which the data is recorded.

In [ ]:
def get_cloud(aoi, date_list):
    """
    Getting all the cloud geometry on `date_list` for `aoi` 

    Method to import all the cloud geometries from local directory to accelerate workflow
    Args:
    - date_list : list of string dates of interest in YYYY-MM-DD format
    - aoi : a single AOI of obesrvations to pull from

    Returns: GeoDataFrame object
    """
    all_cloud_list = []
    all_cloud_date = []
    aoi_data_by_time_dir = "aoi_data_by_time " +  aoi
    aoi_of_interest = aoi
    for date_dir in tqdm.tqdm(os.listdir(aoi_data_by_time_dir)):
      if date_dir in date_list:
        for aoi_dir in os.listdir(aoi_data_by_time_dir + "/" + date_dir):
            if aoi_dir == aoi_of_interest:
                for file in os.listdir(aoi_data_by_time_dir + "/" + date_dir + "/" + aoi_dir):
                    if file == "cloud_cover.shp":
                      all_cloud_list.append(gpd.read_file(aoi_data_by_time_dir + "/" + date_dir + "/" + aoi_dir + "/" + file))
                      all_cloud_date.append(pd.Series(data = [date_dir], index = ['date']))
                      all_cloud_date.append(pd.Series(data = [date_dir], index = ['date']))
                      all_cloud_date.append(pd.Series(data = [date_dir], index = ['date']))
    all_cloud_Aktubinsk = gpd.GeoDataFrame(pd.concat(all_cloud_list, ignore_index = True))
    all_cloud = gpd.GeoDataFrame(pd.concat(all_cloud_date, ignore_index=True)).rename(columns={0: "date"})
    cloud_data = all_cloud_Aktubinsk.join(all_cloud).reset_index()
    return cloud_data

In [ ]:
def get_detection(aoi, date_list):
    """
    Getting all the detections on `date_list` for `aoi` 

    Method to import all the detections from local directory to accelerate workflow
    Args:
    - date_list : list of string dates of interest in YYYY-MM-DD format
    - aoi : a single AOI of obesrvations to pull from

    Returns: GeoDataFrame object
    """
    all_detections_list = []
    aoi_data_by_time_dir = "aoi_data_by_time " + aoi
    all_date = []
    aoi_of_interest = aoi
    for date_dir in tqdm.tqdm(os.listdir(aoi_data_by_time_dir)):
        if date_dir in date_list:
            for aoi_dir in os.listdir(aoi_data_by_time_dir + "/" + date_dir):
                if aoi_dir == aoi_of_interest:
                        for file in os.listdir(aoi_data_by_time_dir + "/" + date_dir + "/" + aoi_dir):
                            if file == "detections.shp":
                                f = gpd.read_file(aoi_data_by_time_dir + "/" + date_dir + "/" + aoi_dir + "/" + file)
                                f['date'] = date_dir
                                all_detections_list.append(f)
    return gpd.GeoDataFrame(pd.concat(all_detections_list, ignore_index=True))

In [ ]:
# get training data

akt_cloud_data_tr = get_cloud('Aktubinsk', train['Aktubinsk'])
akt_det_tr = get_detection('Aktubinsk', train['Aktubinsk'])

mis_cloud_data_tr = get_cloud('Mistrata', train['Misrata Airport'])
mis_det_tr = get_detection('Mistrata', train['Misrata Airport'])

kac_cloud_data_tr = get_cloud('Kacha', train['Kacha airfield'])
kac_det_tr = get_detection('Kacha', train['Kacha airfield'])

tol_cloud_data_tr = get_cloud('Tolmacheva', train['Tolmacheva'])
tol_det_tr = get_detection('Tolmacheva', train['Tolmacheva'])

al_cloud_data_tr = get_cloud('Al-Khadim Air Base', train['Al-Khadim Air Base'])
al_det_tr = get_detection('Al-Khadim Air Base', train['Al-Khadim Air Base'])

### Generate random cloud cover


In [ ]:
def random_points_within(poly, num_points):
  """
  Generate `n` points for the random cloud on AOI `n`

  Args:
  - poly : a polygon object
  - num_points : the number of points from which to construct the cloud

  Returns: Point Geometry Object
  """
  min_x, min_y, max_x, max_y = poly.total_bounds
  points = []
  while len(points) < num_points:
      random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
      points.append(random_point)
  return points

In [ ]:
# dictionary storing AOI names to IDs
aoi_name_to_id = {"Aktubinsk": "e9f9c924-a445-47b8-9e22-8dd802936a89",
                    "Zhukovskiy": "bc3eff13-8a86-486d-aa24-ada1f90e1a8a",
                    "Kacha airfield": "2d892c22-93b7-4fd7-840a-3d8c1172e797",
                    "Tolmacheva": "714b9bb1-a9e4-44ca-be7c-ff4069d52355",
                    "Al Jufra Airbase": "56ceb71e-231a-4df5-8035-17bd1f437db8",
                    "Al-Khadim Air Base": "b496fd94-d64e-4d67-996e-0d0cfb6a5080",
                    "Misrata Airport": "21a34ce3-360d-488c-93af-144696257acc"}


def aoi_with_rand_cc(list_of_aoi, list_of_dates, num_points, all_clouds=akt_cloud_data_tr):  
  """
  Function to generate random cloud covers for inputted AOIs on inputted dates.
  Inputs:
    - list_of_aoi: a list of aoi names (list of strings)
    - list_of_dates: a list of dates to generate the random cloud covers (list of strings)
    - num_points: the number of points our randomly generated cloud cover should have
    - all_clouds: a geodataframe of all the clouds (for the specified aoi)
  
  Returns: `aois` GeoDataFrame containing polygons for each of AOI, Clouds, Clear, and Random (our generated cloud cover)
  """

  AOIs = list_of_aoi # INPUT string list of AOI names
  date_list = list_of_dates # INPUT string list of dates
  
  # convert AOI names to IDs
  aoi_id_to_name = {}
  aois_shply = []
  for key in aoi_name_to_id:
      aoi_id_to_name[aoi_name_to_id[key]] = key
  aoi_ids = [aoi_name_to_id[AOI] for AOI in AOIs]

  aois = all_clouds[all_clouds['date'] == list_of_dates[0]].drop(columns=['index']) # Pull the Geodataframe for the specific aoi on the specific day
  # Below is used to manually derive "clear" since it is often None when imported
  
  # Uses the existing polygon to create a cloud over which we have data
  poly = aois[aois['aoi_id'] == 'AOI']['geometry'].iloc[0]

  # if no clouds, then the clear region is the whole AOI
  if aois[aois['aoi_id'] == 'Clouds']['geometry'].iloc[0]:
    poly = poly.difference(aois[aois['aoi_id'] == 'Clouds']['geometry'].iloc[0].buffer(0))
  full_poly = aois[aois['aoi_id'] == 'AOI']['geometry'] # polygon of the entire AOI
  points = random_points_within(gpd.GeoSeries(poly), num_points)
  rand_poly = Polygon([[p.x, p.y] for p in points]) #  Ensures that the randomly generated polygon is contained within the entire AOI
  
  if len(aois.index) == 4:
      aois['geometry'].iloc[3] = rand_poly # overwrites an existing random cloud
  else:
      aois.loc[len(aois.index)] = ['Random', rand_poly, list_of_dates[0]] # else adds it to the Geodataframe
  return aois

### Generate the Fishnet Grid on AOI

In [ ]:
def fishnetgrid(list_of_aoi, det_tr, all_clouds, list_of_dates, n_cells=20):
  """
  Generates a fishnet grid over the AOIs
  Also converts detections from polygon objects into single points at their centroid
  
  Args:
  - list_of_aoi : a string list of AOI names
  - det_tr : the detections that are part of the training set
  - all_clouds : the data frame of clouds for all the days and AOIs
  - list_of_dates : a string list of dates (YYYY-MM-DD HH:MM:SSZ)

  Options:
  - n_cells : number of cells to divide the grid into

  Returns the grid geodataframe and detections geodataframe
  """
  # AOI DF FOR A CERTAIN DATE AND TIME
  aois = all_clouds[all_clouds['date'] == list_of_dates[0]]
  poly = aois['geometry'].iloc[0]
  xmin, ymin, xmax, ymax = aois.iloc[[0],:].total_bounds

  # projection of the grid
  crs = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs"

  cell_size = (xmax-xmin)/n_cells
  n_cells_y = np.ceil((ymax-ymin)/cell_size)

  # create the cells in a loop
  grid_cells = []
  # Defining Grid Index
  id = []
  for i in np.arange(0, n_cells, 1):
      for j in np.arange(0, n_cells_y+1, 1):
          # bounds
          x1 = xmin+(cell_size*i)
          y1 = ymin+(cell_size*j)
          # Appending to the Grid Index at each iteration
          id.append(np.array([i,j]))
          # Appending to the Grid Cells at each iteration
          grid_cells.append(shapely.geometry.box(x1+cell_size, y1-cell_size, x1, y1)  )
  id = np.array(id)
  idgrid = [i for i in range(len(grid_cells))]
  gridcells = gpd.GeoDataFrame({"id":idgrid,"idx":id[:,0],"idy":id[:,1],"geometry":grid_cells}, crs=crs)
  grid = gridcells[['geometry']]
  # Change detections geodataframe to be centroid points instead of polygons
  det_gdf = det_tr[det_tr['date'] == list_of_dates[0]]['geometry'].centroid #['detections'].iloc[0]['geometry'].centroid
  return grid, det_gdf

def add_cor(grid):
  '''
  Function to add coordinates to grid
  
  Inputs:
  - grid: geodataframe of grid, containing a row for each grid cell, represented by a polygon
  
  Returns: grid geodataframe with the added column `cor` that is the centroid coordinate of each grid cell
  '''
  grid.reset_index(drop=True, inplace=True)

  geom = grid['geometry'] # accesses grid geometry
  cor = [] # initializes coordinates list

  # stores the 0th gridcell centroid coordinate
  x = geom[0].centroid.x
  y = geom[0].centroid.y
  temp = [x,y]
  cor.append(temp)

  # iterate over the remaining centroids of each gridcell
  for i in range(1, len(geom)):
      
      x = geom[i].centroid.x
      y = geom[i].centroid.y
      temp_a = [x,y]
      
      cor.append(temp_a)
      
  grid['cor'] = cor # adds column to gdf
  return grid

### Prediction

Adapted from the simple kriging approach at http://edzer.github.io/mstp/lec5.html


1) `get_prediction` is for a specified latitude & longitude (i.e. for a single gridcell)

2) `get_prediction_grid` is for all the grids in a grid matrix

3) `integrated_prediction` is the function that runs the entire pipeline. This takes in an AOI, the corresponding geometry files, and the date of relevance in order to generate a prediction for each gridcell and return objects that can be plotted in the heatmap

In [ ]:
def get_prediction(grid_matrix, location_lat, location_lon, arr_vehicle):
  '''
  Function to get a prediction (through Kriging) for a specified latitude and longitude
  

  Inputs:
    - grid_matrix: geodataframe of grids, represented in each row by a polygon
    - location_lat: latitude
    - location_lon: longitude
    - arr_vehicle: List of number of vehicles in each grid
  '''
  # for one point with other all other points at the same time 
  a = []
  all_matrices = [] 
  for i in range(len(grid_matrix)):
      a.append([grid_matrix['cor'][i], [location_lat, location_lon]]) # input    
  for it in range(len(a)):
      matrix = np.exp(np.negative(dt.euclidean(a[it][0],a[it][1])))
      all_matrices.append(matrix)
  covariance_matrix = all_matrices
  covariance_matrix_trans = np.array(covariance_matrix).T

  mu = np.mean(arr_vehicle)
  Z_minus_mu = arr_vehicle - mu

  a = []
  w = len(grid_matrix)
  h = len(grid_matrix)
  variance = [[0 for x in range(w)] for y in range(h)] 
  for i in range(len(grid_matrix)):
      for j in range(len(grid_matrix)):
          var = np.exp(np.negative(dt.euclidean(grid_matrix['cor'][i], grid_matrix['cor'][j])))
          variance[i][j] = var

  V_inverse = np.linalg.inv(variance)

  v = all_matrices
  pre = mu + covariance_matrix_trans @ V_inverse @ Z_minus_mu
  return pre

In [ ]:
def get_prediction_grid(grid_matrix, prediction_grid, arr_vehicle):
    '''
    Function to get preductions for all grids in a grid matrix
    Inputs:
      - grid_matrix: gdf of grids, represented in each row by a polygon
      - prediction grid: grids in which we are intersted in seeing the predictions of
      - arr_vehicle: List of number of vehicles in each grid
    '''
    a = []
    for i in range(len(prediction_grid)):
        x = prediction_grid[i][0]
        y = prediction_grid[i][1]
        a.append(get_prediction(grid_matrix, x, y , arr_vehicle))
    return a


This is the function where the `validate` parameter (default = `True`) will generate a random cloud on the input data.

In [ ]:
def integrated_prediction(all_clouds = akt_cloud_data_tr, det_tr = akt_det_tr, AOI = 'Aktubinsk', date ='2020-02-09 08:11:37Z', 
                          num_cells = 30, validate = True, num_rand = 3):
    # Integrated code for 1 AOI and 1 date
    """
    Runs all the above functions to make predictions by gridcell on the AOI
    1. Divdes the AOI into gridcells
    2. Generates a random cloud if validation=True
    3. Computes Kriging prediction for each gridcell
    4. Returns predictions

    Options:
    - all_clouds : cloud training geodataframe for an AOI (default = akt_cloud_data_tr)
    - det_tr : corresponding detections training geodataframe for the AOI (default = akt_det_tr)
    - AOI : string name for the AOI (default = Aktubinsk)
    - date : corresponding string date for which the clouds/detections are observed (default = 2020-02-09 08:11:37Z)
    - num_cells : number of grid cells to divide the AOI into
    - num_rand : number of points to generate a random cloud for validation
    - validate : boolean determining if a random cloud should be generated and predicted for validation (default  = True)

    Returns:
    - grid : fishnet gridcells applied to AOI geometry
    - det_gdf : detections converted to centroid points rather than polygons
    - rand_poly : the geodataframe of geometries including AOI, Clear, Clouds, Random (if validate=True)
    - al_pred_rand_cloud : predicted + baseline (visible) counts for each gridcell
    - arr_vehicle : actual observed counts for each gridcell
    - arr_vehicle_reduced : non-cloudy observed counts for each gridcell (removing cloudy gridcell indices)
    - arr_vehicle_rand : non-cloudy observed counts for each gridcell (zeroing cloudy gridcell indices) -- this is the baseline (visible) counts
    - pred_rand_cloud : predicted counts for cloudy gridcells only
    """

    # applies the fishnet grid
    grid, det_gdf = fishnetgrid([AOI], det_tr = det_tr, all_clouds = all_clouds, list_of_dates = [date], n_cells = num_cells)

    # removes the gridcells that are outside of the AOI
    aois = all_clouds[all_clouds['date'] == [date][0]]
    poly = aois['geometry'].iloc[0]
    indexes_to_remove = []
    for i in range(len(grid)):
        if not grid.iloc[i]['geometry'].intersects(poly):
            indexes_to_remove.append(i)
    grid = grid.drop(grid.index[indexes_to_remove]) # drop the grids that are not touching the aoi at all 

    # OPTIONAL: applies a random cloud if VALIDATE = TRUE
    if validate:
      rand_poly = aoi_with_rand_cc([AOI], [date], num_rand, all_clouds)
      rand_cloud_cover = rand_poly['geometry'].iloc[3]
      max_cloud_percent = .30
      min_cloud_percent = .20
      cloud_percent = rand_cloud_cover.area / poly.area
      while cloud_percent > max_cloud_percent or cloud_percent < min_cloud_percent: # Generate a random cloud and ensure that it is covering less than 30% of the aoi
          rand_poly = aoi_with_rand_cc([AOI], [date], num_rand,all_clouds)
          rand_cloud_cover = rand_poly['geometry'].iloc[3]
          cloud_percent = rand_cloud_cover.area / poly.area

    else:
          # uses real cloud if VALIDATE = FALSE
          rand_poly = aois
          rand_cloud_cover = rand_poly.iloc[0]['geometry']
      
    new_poly = poly.difference(rand_cloud_cover) # aoi polygon subtracting the cloud cover

    reduced_grid = grid.copy()
    # Removing grids that were within the cloud cover
    indexes_to_remove_random_cloud = []
    for i in range(len(reduced_grid)):
        if rand_cloud_cover.contains(reduced_grid.iloc[i]['geometry']):
            indexes_to_remove_random_cloud.append(i)
    reduced_grid = reduced_grid.drop(reduced_grid.index[indexes_to_remove_random_cloud]) # drop the grids that are fully within the cloud cover

    arr_vehicle = [] # nx1 matrix of detections in all grids - this array contains a count of detected objects for every grid
    # this is to validate total predictions
    arr_vehicle_rand = [] # nx1 matrix of detections that do not fall under the cloud - this array contains a count of detected objects for every non-cloud grid
    # arr_vehicle_rand will be equal to arr_vehicle if validate = False because we will have predictions underneath an artificially generated cloud
    # aka this variable will be our baseline (visible) counts
    for polygon in grid['geometry']:
        veh_loc = 0
        veh_rand_loc = 0
        veh_rand_reduced = 0
        for heli in det_gdf:
            if heli.within(polygon):
                veh_loc += 1
                if not heli.within(rand_cloud_cover):
                    veh_rand_loc += 1 
        arr_vehicle.append(veh_loc)
        arr_vehicle_rand.append(veh_rand_loc)

    arr_vehicle_reduced = [] # this is the same as arr_vehicle_rand (i.e. count of detected objects for non-cloud gridcells) except a lower dimension because
    # instead of zeroing out cloudy grid cells, it just skips over them
    for polygon in reduced_grid['geometry']:
        veh_rand_reduced = 0
        for heli in det_gdf:
            if heli.within(polygon) and not heli.within(rand_cloud_cover):
                veh_rand_reduced += 1
        arr_vehicle_reduced.append(veh_rand_reduced)

    # add coordinates to grid and reduced_grid (cloud-less gridcells)
    grid = add_cor(grid)
    reduced_grid = add_cor(reduced_grid)

    pred_rand_cloud = [] # list of predictions within cells intersecting clouds
    al_pred_rand_cloud = arr_vehicle.copy() # list of predictions along with the baseline (visible) counts

    # Getting predictions for grids overlapping the clouded area only
    for i, polygon in grid[grid['geometry'].intersects(rand_cloud_cover)]['geometry'].iteritems():
        var = get_prediction(reduced_grid, polygon.centroid.x, polygon.centroid.y, arr_vehicle_reduced)
        pred_rand_cloud.append(var)
        al_pred_rand_cloud[i] = var

    # Round predictions to 3 decimal places
    pred_rand_cloud = [round(i,3) for i in pred_rand_cloud]
    al_pred_rand_cloud = [round(i,3) for i in al_pred_rand_cloud]

    return grid, det_gdf, rand_poly, al_pred_rand_cloud, arr_vehicle, arr_vehicle_reduced, arr_vehicle_rand, pred_rand_cloud

# Make predictions

In [ ]:
# In this example, we use Aktubinsk
AOI_name = 'Aktubinsk'
detections_df = akt_det_tr
geometries_df = akt_cloud_data_tr
generate_clouds = True # set this to false for unseen data (i.e. to not validate)

In [ ]:
pred_list = []
counter = 0

for date in tqdm.tqdm(np.unique(detections_df['date'].values)):
  grid, det_gdf, rand_poly, al_pred_rand_cloud, arr_vehicle, arr_vehicle_reduced, arr_vehicle_rand, pred_rand_cloud = integrated_prediction(all_clouds = geometries_df, 
                                                                                                                                            det_tr = detections_df, AOI = AOI_name, 
                                                                                                                                            date = date, num_cells = 30, num_rand = 3,
                                                                                                                                            validate = generate_clouds)
  # pred_list contains [date, actual observed counts, predicted counts = baseline + predictions, baseline (visible) counts]
  pred_list.append([date, sum(arr_vehicle), round(sum(al_pred_rand_cloud)), sum(arr_vehicle_rand)])

pred_list[:5]
    

## Prediction metrics and visualization

In [ ]:
def get_mape(y_true, y_pred):
  """
  Return the Mean Absolute Percentage Error (MAPE) for a given set of predictions
  Args:
  - y_true : the true counts of detected objects
  - y_pred : the predicted counts of detected objects
  Output: float
  """
  mape = mean_absolute_percentage_error(y_true, y_pred)
  return mape

In [ ]:
### Generating Mean Absolute Percentage Error for each AOI across all days.
get_mape([a[1] for a in pred_list], [a[2] for a in pred_list])

In [ ]:
def timeseries_pred_plot(AOI_name, pred_list, detections_df):
  """  
  Prints out the Mean Absolute Percentage Error (MAPE) for a set of predictions
  and also generates a time series plot of the actual, predicted, and visible
  detections over a set of dates 

  Args:
    - pred_list : a 4D array with columns [dates, actual_counts, predicted_counts, visible_counts]

  Output: None, shows Matplotlib figure
  """
  y_true = [a[1] for a in pred_list]
  y_pred = [a[2] for a in pred_list]
  y_base = [a[3] for a in pred_list]
  dates = [a[0] for a in pred_list]

  mape = get_mape(y_true, y_pred)
  print(f'Mean absolute percentage error: {mape}')

  plt.figure(figsize=(10, 5))
  plt.plot(dates, y_true, label = "actual detections")
  plt.plot(dates, y_pred, label = "predicted detections")
  plt.plot(dates, y_base, label = "baseline (visible)", color='green')
  plt.xticks(rotation=90)
  plt.legend()
  plt.xticks(" ")

  date_end = max(detections_df['date'].values)
  date_start = min(detections_df['date'].values)

  plt.title(f"Kriging Prediction Plots for {AOI_name} from {date_start} to {date_end}")
  plt.show()

In [ ]:
timeseries_pred_plot(AOI_name, pred_list, detections_df)

The green line (visible) line will be identical to the observed (blue) line if no random cloud was generated.

In [ ]:
### sanity checks
# arr_vehicle: List of detections for all grids in AOI
# arr_vehicle_rand: List of detections for all grids in AOI that aren't covered by randomly generated cloud
# arr_vehicle_reduced: List of detections for not-completely-cloud-covered grids in AOI whose detections aren't covered by cloud
###

print("sum arr_vehicle_reduced:", sum(arr_vehicle_reduced))
print("sum arr_vehicle:", sum(arr_vehicle))
print("sum arr_vehicle_rand:", sum(arr_vehicle_rand))
print("len arr_vehicle_reduced:", len(arr_vehicle_reduced))
print("len arr_vehicle:", len(arr_vehicle))
print("len arr_vehicle_rand:", len(arr_vehicle_rand))

In [ ]:
# Comparing kriging detections to actual detections
import itertools
import operator

print("Correct number of detections in generated cloud cover:", sum(arr_vehicle))
print("Predicted counts in generated cloud cover:", sum(al_pred_rand_cloud))
print("Baseline counts which were visible:", sum(arr_vehicle_rand))

## Plotting into a heatmap 

In [ ]:
def heatmap(grid, det_gdf, rand_poly, al_pred_rand_cloud, arr_vehicle, arr_vehicle_reduced, 
            arr_vehicle_rand, pred_rand_cloud,
            AOI_name, geometries_df, detections_df):
    """
    Plots a heatmap overalying the actual and predicted counts onto the AOI, including cloud cover

    Args:
    The arguments are all the outputs of the `integrated_prediction` function along with some

    (`integrated_prediction` function outputs)
    - grid : fishnet gridcells applied to AOI geometry
    - det_gdf : detections converted to centroid points rather than polygons
    - rand_poly : the geodataframe of geometries including AOI, Clear, Clouds, Random (if validate=True)
    - al_pred_rand_cloud : predicted + baseline (visible) counts for each gridcell
    - arr_vehicle : actual observed counts for each gridcell
    - arr_vehicle_reduced : non-cloudy observed counts for each gridcell (removing cloudy gridcell indices)
    - arr_vehicle_rand : non-cloudy observed counts for each gridcell (zeroing cloudy gridcell indices) -- this is the baseline (visible) counts
    - pred_rand_cloud : predicted counts for cloudy gridcells only

    (other inputs)
    - AOI_name : string name of AOI
    - geometries_df : the geodataframe with all the geometries for the AOI across all the training dates
    - detections_df : the geodataframe wtih all the detections for the AOI across all the training dates

    Output: None, shows Matplotlib figure
    """
    # Plotting output into a heat map for a specific date and AOI

    prediction_grid = grid['cor']
    grid['Prediction_all'] = al_pred_rand_cloud
    grid['Detected_hidden_by_rand_cloud'] = arr_vehicle_rand
    grid['Detected'] = arr_vehicle
    grid['Detected_hidden_by_rand_cloud'] = grid['Detected_hidden_by_rand_cloud'].astype(np.float64)
    grid['Detected'] = grid['Detected'].astype(np.float64)

    # plot the cloud
    cloud_gdf = rand_poly[rand_poly['aoi_id'] == 'Clear']
    random_gdf = rand_poly[rand_poly['aoi_id'] == 'Random'] # if validate = True

    aois = geometries_df[geometries_df['date'] == detections_df['date'].values[0]]
    poly = [aois['geometry'].iloc[0]]
    type(poly)
    poly_gdf = gpd.GeoDataFrame(poly)
    poly_gdf = poly_gdf.rename({0:'geometry'}, axis = 1)


    # heatmap
    base = grid.plot(figsize=(30, 10), alpha=0.5, legend = True, edgecolor = 'black', column='Prediction_all', scheme= 'quantiles', k=3, cmap='Blues')
    poly_gdf.plot(ax= base, edgecolor = 'black', color = 'yellow', alpha=0.2)
    cloud_gdf.plot(ax= base, color = 'red', alpha = 0.3)
    random_gdf.plot(ax = base, color = 'green', alpha = 0.3) # if validate = True
    plt.title(f'Heatmap for predictions on AOI {AOI_name}')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")

    for i in range(len(grid)):
        xy_val = tuple(grid['cor'][i])
        value = round(grid['Prediction_all'][i])
        plt.annotate(value, xy=xy_val, color = 'green', xytext=(xy_val[0]+0.00005, xy_val[1]+0.0005), fontsize = 8)
        xy_detected_cloud = grid['Detected_hidden_by_rand_cloud'][i]
        xy_detected = grid['Detected'][i]
        plt.annotate(xy_detected, xy=xy_val, color = 'black', xytext=(xy_val[0]-0.0003, xy_val[1]-0.0003), fontsize = 8)


    from matplotlib.lines import Line2D
    custom_lines = [Line2D([0], [0], color='black', lw=4),
                    Line2D([0], [0], color='green', lw=4)]
    plt.legend(custom_lines, ['Actual counts', 'Predicted'])

    plt.show()

In [ ]:
heatmap(grid, det_gdf, rand_poly, al_pred_rand_cloud, arr_vehicle, arr_vehicle_reduced, 
            arr_vehicle_rand, pred_rand_cloud,
            AOI_name, geometries_df, detections_df)